In [3]:
import os
import pandas as pd
from PyPDF2 import PdfReader

In [4]:
def extract_text_from_pdfs(directory_path):
    data = {'filename': [], 'text': []}
    
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            file_path = os.path.join(directory_path, filename)
            
            # Extract text from PDF
            text = ""
            with open(file_path, "rb") as pdf_file:
                pdf_reader = PdfReader(pdf_file)
                for page in pdf_reader.pages:
                    text += page.extract_text() or ""
                    
            # Append filename and text to data
            data['filename'].append(filename)
            data['text'].append(text)
    
    # Create DataFrame
    df = pd.DataFrame(data)
    return df

In [5]:
# Usage
directory_path = "data/"
df = extract_text_from_pdfs(directory_path)

                                           filename  \
0             IGA 451M - 2020 Spring 1 (170371).pdf   
1               STAT 123 - 2022 Spring (126048).pdf   
2               STAT 364 - 2022 Spring (214539).pdf   
3                 PSY 2030 - 2021 Fall (160667).pdf   
4                EPI 224 - 2024 Fall 2 (190291).pdf   
..                                              ...   
244               ECON 970 - 2024 Fall (122752).pdf   
245               STAT 139 - 2024 Fall (110751).pdf   
246               STAT 221 - 2020 Fall (115077).pdf   
247  BIOSTAT 241_BST 241 - 2024 Spring (190066).pdf   
248       SYSBIO 220QC - 2020 Spring 1 (204480).pdf   

                                                  text  
0    IGA-451M: CONTROVERSIES IN CLIMATE, ENERGY \n&...  
1    Thank\tyou\tfor\ttaking\tStat\t123\t2022,\tQua...  
2    Stat364 : Scalable Statistical Inference for B...  
3    Bayesian Data Analysis\nPsych 2030 { Fall 2021...  
4     \nEPI224 Cancer Prevention (2023 Fall 2)  Syl...

In [20]:
import re
df['cleaned_text'] = df['text'].apply(lambda x: re.sub(r'\s+', ' ', x))

In [21]:
df

,filename,text,cleaned_text
0,IGA 451M - 2020 Spring 1 (170371).pdf,"IGA-451M: CONTROVERSIES IN CLIMATE, ENERGY \n&...","IGA-451M: CONTROVERSIES IN CLIMATE, ENERGY & T..."
1,STAT 123 - 2022 Spring (126048).pdf,"Thank\tyou\tfor\ttaking\tStat\t123\t2022,\tQua...","Thank you for taking Stat 123 2022, Quantitati..."
2,STAT 364 - 2022 Spring (214539).pdf,Stat364 : Scalable Statistical Inference for B...,Stat364 : Scalable Statistical Inference for B...
3,PSY 2030 - 2021 Fall (160667).pdf,Bayesian Data Analysis\nPsych 2030 { Fall 2021...,Bayesian Data Analysis Psych 2030 { Fall 2021 ...
4,EPI 224 - 2024 Fall 2 (190291).pdf,\nEPI224 Cancer Prevention (2023 Fall 2) Syl...,EPI224 Cancer Prevention (2023 Fall 2) Syllab...
...,...,...,...
244,ECON 970 - 2024 Fall (122752).pdf,Economics\t970\n\t\nis\ta\tseries\tof\tseminar...,Economics 970 is a series of seminars taught i...
245,STAT 139 - 2024 Fall (110751).pdf,STAT 139: Introduction to Linear Models\nFall ...,STAT 139: Introduction to Linear Models Fall 2...
246,STAT 221 - 2020 Fall (115077).pdf,Fall 2020\nSTAT 221: Computational Tools for S...,Fall 2020 STAT 221: Computational Tools for St...
247,BIOSTAT 241_BST 241 - 2024 Spring (190066).pdf,BST 241/ BIOSTAT 241: Inference II\n Spring 2...,BST 241/ BIOSTAT 241: Inference II Spring 202...


In [22]:
df.to_csv('processed_data/stat_courses.csv', index = False)